In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import numpy as np
from tqdm import tqdm
from typing import Tuple
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np
import sys
import math
from typing import Tuple

class Node:
    def __init__(self, feature_index: int = None, threshold: float = None, value: int = None, left: 'Node' = None, right: 'Node' = None):
        self.feature_index=feature_index
        self.threshold=threshold
        self.value=value
        self.left=left
        self.right=right

class DecisionTreeRegressor:
    def __init__(self):
        pass

    def fit(self, X: np.ndarray, y: np.ndarray):
        self.n_classes = len(set(y))
        self.n_features = X.shape[1]
        self.tree = self._grow_tree(X, y)

    def _grow_tree(self, X: np.ndarray, y: np.ndarray, depth: int = 0):
        if depth == 1:
            return Node(value=np.mean(y))

        best_index, best_threshold = self._find_best_split(X, y)

        left_indices, right_indices = self._split(X[:, best_index], best_threshold)

        left_node = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
        right_node = self._grow_tree(X[right_indices], y[right_indices], depth + 1)

        return Node(feature_index=best_index, threshold=best_threshold, left=left_node, right=right_node)

    def _find_best_split(self, X: np.ndarray, y: np.ndarray):
        best_SSR=sys.float_info.max
        best_feature, best_threshold=None, None

        for feature_index in range(self.n_features):
            sorted_indices=np.argsort(X[:,feature_index])
            sorted_x=X[sorted_indices]
            # sorted_y=Y[sorted_indices]
            tim=0
            thresholds=np.unique(sorted_x[:, feature_index])
            for threshold_index in range(len(thresholds)-1):
                threshold=(thresholds[threshold_index]+thresholds[threshold_index+1])/2
                left_indice,right_indice=self._split(X[:, feature_index], threshold)
                ssr=self._total_SSR(y[left_indice], y[right_indice])
                if(ssr<best_SSR):
                    best_SSR=ssr
                    best_feature=feature_index
                    best_threshold=threshold

        return best_feature, best_threshold


    def _calc_SSR(self, y : np.ndarray):
        y_avg=np.mean(y)
        return np.sum((y-y_avg)**2)

    def _total_SSR(self, y_left:np.ndarray, y_right:np.ndarray):
        left_SSR=self._calc_SSR(y_left)
        right_SSR=self._calc_SSR(y_right)
        return (left_SSR+right_SSR)

    def _split(self, feature: np.ndarray, threshold:float ):
        left_indices = np.where(feature <= threshold)[0]
        right_indices = np.where(feature > threshold)[0]
        return left_indices, right_indices

    def predict(self, X : np.ndarray):
         return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _traverse_tree(self, X : np.ndarray, node: Node):
        if(node.left is None and node.right is None):
            return node.value
        if X[node.feature_index] <= node.threshold:
            return self._traverse_tree(X, node.left)
        else:
            return self._traverse_tree(X, node.right)

class GradientBoost:
    def __init__(self, training_rate : float=1):
        self.trees=[]
        self.residuals=None
        self.training_rate=training_rate
        self.losses=[]
        self.y_mean=None

    def __take_gradient(self, y, y_pred):
        grad = -(y-y_pred)
        return grad

    def _add_tree(self, X : np.ndarray, Y  : np.ndarray):
        if(len(self.trees)==0):
            self.y_mean=np.mean(Y)
            self.residuals=np.full(X.shape[0], self.y_mean)

        tree=DecisionTreeRegressor()
        grads=self.__take_gradient(Y, self.residuals)
        tree.fit(X,grads)
        self.trees.append(tree)
        self.residuals-=self.training_rate*(tree.predict(X))

    def predict(self, X:np.ndarray):
        pred=np.full(X.shape[0], self.y_mean)
        for tree in self.trees:
            pred-=self.training_rate*(tree.predict(X))
        return pred








QUESTION 2

In [ ]:
import numpy as np
from tqdm import tqdm
from typing import Tuple
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

def main():
    data = np.load('/content/drive/MyDrive/SML_A4/mnist.npz')
    x_train = data['x_train']
    y_train = data['y_train']
    x_test = data['x_test']
    y_test = data['y_test']

    selected_classes=[0,1]
    train_mask=np.isin(y_train, selected_classes)
    x_train=x_train[train_mask]
    y_train=y_train[train_mask]

    test_mask=np.isin(y_test, selected_classes)
    x_test=x_test[test_mask]
    y_test=y_test[test_mask]

    x_train=x_train.reshape(x_train.shape[0], -1)
    x_test=x_test.reshape(x_test.shape[0], -1)

    x_train, x_val, y_train, y_val=train_test_split(x_train, y_train, test_size=1000, stratify=y_train, random_state=42)

    PCA_dim=5
    pca=PCA(n_components=PCA_dim)
    x_train=pca.fit_transform(x_train)
    x_val=pca.transform(x_val)
    x_test=pca.transform(x_test)

    tree=GradientBoost(training_rate=0.01)

    epoch=300
    for _ in range(epoch):
        tree._add_tree(x_train, y_train)
        y_val_pred=tree.predict(x_val)
        print(f"Validation MSE: {np.mean((y_val-y_val_pred)**2)}")

    y_pred=tree.predict(x_test)
    print(f"Test MSE: {np.mean((y_test-y_pred)**2)}")


if __name__== '__main__':
    main()

Validation MSE: 0.24411788737442988
Validation MSE: 0.23935634940175046
Validation MSE: 0.23468959225176034
Validation MSE: 0.23011572952391748
Validation MSE: 0.2256329123596728
Validation MSE: 0.22123932869535753
Validation MSE: 0.21693320252993933
Validation MSE: 0.21271279320735045
Validation MSE: 0.20857639471309727
Validation MSE: 0.2045223349848677
Validation MSE: 0.20054897523685794
Validation MSE: 0.19665470929754536
Validation MSE: 0.1928379629606398
Validation MSE: 0.18909719334895117
Validation MSE: 0.18543088829091658
Validation MSE: 0.1818375657095345
Validation MSE: 0.17831331849593948
Validation MSE: 0.17486132647788186
Validation MSE: 0.17147564733002646
Validation MSE: 0.16816514496057988
Validation MSE: 0.16491235870338983
Validation MSE: 0.16173197165936995
Validation MSE: 0.15860686486046882
Validation MSE: 0.15555148524871443
Validation MSE: 0.15254904894484314
Validation MSE: 0.14961376926987954
Validation MSE: 0.1467291909487842
Validation MSE: 0.143909296253027